# Using SWIG with the Point class

Important! The `CompPhys` git directory must be parallel to this one. So it will look something like this: 

```
current directory (/results)
   |
   |-- /results/CompPhys
   |
   |-- /results/technical-assignment-5-rappoccio
```

We will reuse code from `CompPhys`.

## The Point class

Here, we reuse the Point class that we created in CompPhys. If you want to remind yourself, uncomment the following two commands

In [12]:
! cat ../CompPhys/ReviewCpp/InheritanceExample/Point3d.h

#ifndef Point3d_h
#define Point3d_h

#include <iostream>
#include <fstream>
#include "Point.h"

class Point3d : public Point {
 public:
  Point3d( double ix=0., double iy=0., double iz=0.);
  virtual ~Point3d();

  // Must be declared as a virtual override
  virtual void print(std::ostream & out=std::cout) const override;

  // x() and y() accessors are available through the base class Point
  double z() const;

  // Need to override all of these:
  Point3d operator+( Point3d const & right ) const ;
  Point3d operator-( Point3d const & right ) const ;
  Point3d & operator+=( Point3d const & right ) ;
  Point3d & operator-=( Point3d const & right ) ;

  // Must be declared as a virtual override
  virtual bool input( std::istream & in) override;

 private: 
  double z_;

  // Doubles x_ and y_ are in the base class!

};

#endif


In [2]:
! cat ../CompPhys/ReviewCpp/ClassExample/Point.cc

#include "Point.h"


Point::Point( double ix, double iy) { 
  x_=ix;y_=iy;
}

Point::~Point(){}

void Point::print() const {
  std::cout << "(" << x_ << "," << y_ << ")" << std::endl;
};

double Point::x() const { return x_;}
double Point::y() const { return y_;}

Point Point::operator+( Point const & right ) const {
  Point retval( x_ + right.x_, y_ + right.y_ );
  return retval;
}

Point Point::operator-( Point const & right ) const {
  Point retval( x_ - right.x_, y_ - right.y_ );
  return retval;
}

Point & Point::operator+=( Point const & right )  {
  x_ += right.x_; y_ += right.y_ ;
  return *this;
}

Point & Point::operator-=( Point const & right )  {
  x_ -= right.x_; y_ -= right.y_ ;
  return *this;
}

bool Point::input( std::istream & in ) {

  std::string line;   
  std::getline( in, line, ',');   
  x_ = std::atof( line.c_str() ); 
  std::getline( in, line );   
  y_ = std::atof( line.c_str() );       
  if ( line == "") {      
    return false;  
  } 
  else {
    return 

## The swig files

Here, we're going to just include the `"Point.h"` file from above with the right syntax. Then we have a `"setup.py"` file to describe how to compile it. 

In [3]:
cat swig_point/point.i

%module point
/* First: Include your own code.*/
%{
#include "CompPhys/ReviewCpp/ClassExample/Point.h"
%}
/* Next: get the class information*/

%include "CompPhys/ReviewCpp/ClassExample/Point.h"


In [4]:
cat swig_point/setup.py

#!/usr/bin/env python

"""
setup.py file for SWIG example
"""

from distutils.core import setup, Extension


point_module = Extension('_point',
                           sources=['../CompPhys/ReviewCpp/ClassExample/Point.cc', 'swig_point/point_wrap.cxx' ],
                           extra_compile_args=["-std=c++11", "-I../"],
                           )

setup (name = 'point',
       version = '0.1',
       author      = "SWIG Docs",
       description = """Simple swig point from docs""",
       ext_modules = [point_module],
       py_modules = ["point"],
       )


## Create the interface with SWIG

Note! We have a function called `"print"`, which is a keyword, so we cannot use that. It will be renamed to `"_print"` with an underscore in the front. 

In [5]:
! swig -c++ -python -I../ swig_point/point.i

../CompPhys/ReviewCpp/ClassExample/Point.h:11: Warning 314: 'print' is a python keyword, renaming to '_print'


## Here's the compilation of the files

In [6]:
! python swig_point/setup.py build_ext --inplace

running build_ext
building '_point' extension
creating build
creating build/CompPhys
creating build/CompPhys/ReviewCpp
creating build/CompPhys/ReviewCpp/ClassExample
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/swig_point
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c ../CompPhys/ReviewCpp/ClassExample/Point.cc -o build/temp.linux-x86_64-3.7/../CompPhys/ReviewCpp/ClassExample/Point.o -std=c++11 -I../
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c swig_point/point_wrap.cxx -o build/temp.linux-x86_64-3.7/swig_point/point_wrap.o -std=c++11 -I../
x86_64-linux-gnu-g++ -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-Bsymbolic-functions -Wl,-z

## Next setup the path

We need to tell python where to find the libraries we just compiled. 

In [7]:
import sys
import os

sys.path.append( os.path.abspath("swig_point") )

print (sys.path)




['/results/CompPhysNumpyExamples', '/app/compphys_python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/results/.ipython', '/results/CompPhysNumpyExamples/swig_point']


## Ready!

We have the `Point` class (uppercase "P") in module "point" (lowercase "p"). 


In [8]:
from point import Point

## Get help

You can see what was generated by asking for help! 

In [9]:
help("point")

Help on module point:

NAME
    point

DESCRIPTION
    # This file was automatically generated by SWIG (http://www.swig.org).
    # Version 3.0.12
    #
    # Do not make changes to this file unless you know what you are doing--modify
    # the SWIG interface file instead.

CLASSES
    builtins.object
        Point
    
    class Point(builtins.object)
     |  Point(ix=0.0, iy=0.0)
     |  
     |  Methods defined here:
     |  
     |  __add__(self, right)
     |  
     |  __del__ lambda self
     |  
     |  __getattr__ lambda self, name
     |  
     |  __iadd__(self, right)
     |  
     |  __init__(self, ix=0.0, iy=0.0)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  __isub__(self, right)
     |  
     |  __repr__ = _swig_repr(self)
     |  
     |  __setattr__ lambda self, name, value
     |  
     |  __sub__(self, right)
     |  
     |  input(self, arg2)
     |  
     |  x(self)
     |  
     |  y(self)
     |  
     |  --------------

## Do some math

Here we can use our `operator+` class within python! Woohoo!

In [10]:
p1 = Point(1,2)
p2 = Point(2,3)
p3 = p1 + p2

In [11]:
print(p3.x())

3.0


# Problem 3: SWIG for Point3d


## Problem 3a
a. You will now repeat the steps above for the `Point3d` class in an interface called `point3d`. You are free to use the `Point3d` class with the inheritance as defined in the `CompPhys/ReviewCpp/InheritanceExample` folder, or write a simpler one. 

- Create `swig_point/point3d.i`
- Create `swig_point/setup3d.py`
- Create the interface with the `swig` command. 
- Compile with `setup3d.py` similarly to above.
- Import `point3d`, create two 3-d points `v1` and `v2`, set them to `(1,2,3)` and `(4,5,6)`. Set a third point (`v3`) equal to their sum. 



In [ ]:
### YOUR CODE GOES HERE. 

## Problem 3b


b. Write another file called `pointops` that will implement operations on `Point3d`. Define a function to compute the cross product of the two vectors with the following C++ interface:

```
Point3d cross( Point3d const & a, Point3d const & b);
```
- Create `swig_point/cross.i`
- Create `swig_point/setup_cross.py`
- Create the interface with the `swig` command. 
- Compile with `cross.py` similarly to above.
- Then take the cross product of 3-d points `v1` and `v2` above. 

In [ ]:
### YOUR CODE GOES HERE. 